In [ ]:
%config Completer.use_jedi = False
%load_ext autoreload
%autoreload 2

## Submission

In [ ]:
from typing import List, Tuple

import hydra
import lightning as pl
from omegaconf import DictConfig
from lightning import Callback, LightningDataModule, LightningModule, Trainer
from lightning.pytorch.loggers import Logger

from terralearn import utils

import torch
import hydra
import omegaconf
import pyrootutils
import pandas as pd
from tqdm import tqdm
import numpy as np
import pickle
import timm
import matplotlib.pyplot as plt

trait_columns = [
    "X4_mean",
    "X11_mean",
    "X18_mean",
    "X50_mean",
    "X26_mean",
    "X3112_mean",
]
sub_cols = [i.replace("_mean", "") for i in trait_columns]
from sklearn.metrics import mean_squared_error, r2_score
from torchmetrics.regression import R2Score
from fgvc.models.plant_traits_model import *
from torchmetrics.functional import r2_score
from glob import glob

### Ensemble

In [ ]:
df_names = [
    ("v0_reg.csv", 1),
    ("v0_1_reg.csv", 1),
    ("v1_reg.csv", 1),
    ("v1_clf.csv", 1),
    # ("v1_1_reg.csv", 2),
    # ("v1_1_clf.csv", 2),
    # ("v1_2_reg.csv", 2),
    # ("v1_2_clf.csv", 2),
    # ("v1_3_reg.csv", 1),
    # ("v1_3_clf.csv", 1),
    # ("v1_3_bld.csv", 1),
    # ("v1_4_reg.csv", 1),
    # ("v1_4_clf.csv", 1),
    # ("v1_4_bld.csv", 1),
    # ("v1_5_reg.csv", 1),
    # ("v1_5_clf.csv", 1),
    # ("v1_5_bld.csv", 1),
    ]

# load the csvs and take weighted average of the predictions for the sub_cols columns and out put final csv
out = pd.DataFrame()
weight_sum = 0
for filename, weight in df_names:
    df = pd.read_csv(filename)
    if len(out) != 0:
        out += df[sub_cols] * weight
    else:
        out = df[sub_cols] * weight
    weight_sum += weight

out = out / weight_sum
df_avg = pd.DataFrame()
df_avg["id"] = df["id"]
df_avg[sub_cols] = out

In [ ]:
df_avg

In [ ]:
df_avg.to_csv("avg.csv", index=False)

In [ ]:
!kaggle competitions submit -c planttraits2024 -f avg.csv -m " avg"

In [ ]:
sorted(glob("v*.csv"))

In [ ]:
# df1 = pd.read_csv("v0_reg.csv")
# df2 = pd.read_csv("v0_1_reg.csv")
df3 = pd.read_csv("v1_1_reg.csv")
df4 = pd.read_csv("v1_1_clf.csv")
df5 = pd.read_csv("v1_2_reg.csv")
df6 = pd.read_csv("v1_2_clf.csv")
df3[sub_cols] = df3[sub_cols] + df4[sub_cols] + df5[sub_cols] + df6[sub_cols]
df3[sub_cols] = df3[sub_cols] / 4
df3.to_csv("avg.csv", index=False)

In [ ]:
df1

In [ ]:
df2

In [ ]:
!kaggle competitions submit -c planttraits2024 -f avg.csv -m " mean v1_1 v1_2"

In [ ]:
df_out = df[0]
for df_each in df[1:]:
    # sum the trait columns
    df_out[sub_cols] += df_each[sub_cols]
# take average
df_out[sub_cols] /= len(df)
    


In [ ]:
df_out

In [ ]:
df_out.to_csv("avg_v0_v_1_3.csv", index=False)

In [ ]:
!kaggle competitions submit -c planttraits2024 -f v_1_2_avg.csv -m "v_1_2_avg"

In [ ]:
sub = pd.read_csv('avg_5_best.csv')
print(r2_score(torch.tensor(sub[sub.columns[1:]].values), torch.tensor(df_out[sub.columns[1:]].values)))

In [ ]:
df1 = pd.read_csv("v1_2_reg.csv")
df2 = pd.read_csv("v1_2_clf.csv")
df1[sub_cols] = df1[sub_cols] + df2[sub_cols] 
df1[sub_cols] = df1[sub_cols] / 2
df1.to_csv("v_1_2_avg.csv", index=False)

In [ ]:
a, b, a * b, a+a

### Setup

In [ ]:
cfg = omegaconf.OmegaConf.load("/home/ubuntu/FGVC11/configs/data/plant_traits_data.yaml")
cfg.batch_size = 128
datamodule: LightningDataModule = hydra.utils.instantiate(cfg)
datamodule.setup()

In [ ]:
# data = datamodule.data_train[11903]
# plt.imshow(data["image"].permute(1, 2, 0))
# plt.show()
# print(data["label"])
# print(data["original_label"])
# print(data["aux_label"])

In [ ]:
# (len(datamodule.data_train)//(64*4))*30
device = 'cuda:0'

In [ ]:
cfg = omegaconf.OmegaConf.load("/home/ubuntu/FGVC11/configs/model/plant_traits_model.yaml")
model = hydra.utils.instantiate(cfg)



model = model.load_from_checkpoint("/home/ubuntu/FGVC11/logs/train/runs/blk_4_blended_traits_simple_mixup/checkpoints/epoch_101.ckpt");
model = model.train();
model = model.eval();
model = model.to(device);

In [ ]:
# ??model.model.body.forward_head

In [ ]:
# for name, param in model.named_parameters():
#     if param.requires_grad:
#         print(name)


### On test data

In [ ]:
df_test = pd.read_csv('/home/ubuntu/FGVC11/data/PlantTrait/test.csv')
df_test['path'] = '/home/ubuntu/FGVC11/data/PlantTrait/test_images/' + df_test['id'].astype(str) + '.jpeg'

In [ ]:
model.reg_weight, model.clf_weight

In [ ]:
reg_pred = []
clf_pred = []
bld_pred = []
i = 0
for batch in tqdm(datamodule.test_dataloader()):
    # Unpack the batch
    x, x_ = batch["image"].to(device), batch["metadata"].to(device)
    # Move data to the device

    # Make predictions
    with torch.no_grad():
        pred_enc, specie_logits = model.model.forward_alt(x, x_)
        pred = model.model.le.inverse_transform(pred_enc.clone().detach())

        pred_specie = torch.argmax(specie_logits, dim=1)
        pred_specie_traits = model.specie_traits[pred_specie]
        # Append predictions to the list
        reg_pred.append(pred.cpu())
        clf_pred.append(pred_specie_traits.cpu())

        if model.blend_traits:
            blended_traits = (
                model.reg_weight * pred + model.clf_weight * pred_specie_traits
            ) / (model.reg_weight + model.clf_weight)
            # pred_clf_enc = model.model.le.transform(pred_specie_traits)
            # blended_traits_enc = model.trait_blender([pred_enc, pred_clf_enc])
            # blended_traits = model.model.le.inverse_transform(blended_traits_enc)
            bld_pred.append(blended_traits.cpu())

# Concatenate predictions from all batches
reg_pred = torch.concat(reg_pred, dim=0)
clf_pred = torch.concat(clf_pred, dim=0)
if model.blend_traits:
    bld_pred = torch.concat(bld_pred, dim=0)
# all_tgts = torch.concat(all_tgts, dim=0)

In [ ]:
reg_pred = reg_pred.numpy()
clf_pred = clf_pred.numpy()
if model.blend_traits:
    bld_pred = bld_pred.numpy()

In [ ]:
# Create a DataFrame with the predictions and corresponding IDs
reg_df = pd.DataFrame({
    'id': df_test['id'].values,
    'X4': reg_pred[:, 0],
    'X11': reg_pred[:, 1],
    'X18': reg_pred[:, 2],
    'X50': reg_pred[:, 3],
    'X26': reg_pred[:, 4],
    'X3112': reg_pred[:, 5],
})
# Create a DataFrame with the predictions and corresponding IDs
clf_df = pd.DataFrame({
    'id': df_test['id'].values,
    'X4': clf_pred[:, 0],
    'X11': clf_pred[:, 1],
    'X18': clf_pred[:, 2],
    'X50': clf_pred[:, 3],
    'X26': clf_pred[:, 4],
    'X3112': clf_pred[:, 5],
})
if model.blend_traits:
    bld_df = pd.DataFrame({
        'id': df_test['id'].values,
        'X4': bld_pred[:, 0],
        'X11': bld_pred[:, 1],
        'X18': bld_pred[:, 2],
        'X50': bld_pred[:, 3],
        'X26': bld_pred[:, 4],
        'X3112': bld_pred[:, 5],
    })
# submission_df.to_csv('submission.csv', index=False)
sub = pd.read_csv('avg_5_best.csv')
print(r2_score(torch.tensor(sub[sub.columns[1:]].values), torch.tensor(reg_df[sub.columns[1:]].values)))
print(r2_score(torch.tensor(sub[sub.columns[1:]].values), torch.tensor(clf_df[sub.columns[1:]].values)))
if model.blend_traits:
    print(r2_score(torch.tensor(sub[sub.columns[1:]].values), torch.tensor(bld_df[sub.columns[1:]].values)))

In [ ]:
# comb_df = (reg_df + clf_df)/2
# comb_df['id'] = reg_df['id']
# print(r2_score(torch.tensor(sub[sub.columns[1:]].values), torch.tensor(comb_df[sub.columns[1:]].values)))

In [ ]:
reg_df.to_csv('v1_5_reg.csv', index=False)
clf_df.to_csv('v1_5_clf.csv', index=False)
if model.blend_traits:
    bld_df.to_csv('v1_5_bld.csv', index=False)

In [ ]:
bld_df

In [ ]:
# best = pd.read_csv('best.csv')
# edit = best.copy()
# for i in range(len(best)):
#     a = best.iloc[i].values[1:]
#     b = clf_df.iloc[i].values[1:]
#     score = r2_score(torch.tensor(a), torch.tensor(b))
#     if score > 0.7:
#         edit.iloc[i, 1:] = clf_df.iloc[i].values[1:]
# edit.to_csv('edit.csv', index=False)

In [ ]:
!kaggle competitions submit -c planttraits2024 -f v1_5_bld.csv -m "v1_5_bld"

In [ ]:
!kaggle competitions submit -c planttraits2024 -f v1_5_reg.csv -m "v1_5_reg"

In [ ]:
!kaggle competitions submit -c planttraits2024 -f v1_5_clf.csv -m "v1_5_clf"

### Run Inference on all

In [ ]:
from fgvc.data.plant_traits_data import PlantTraitsDataset
from torch.utils.data import DataLoader

In [ ]:
df_complete = pd.read_csv("/home/ubuntu/FGVC11/data/PlantTrait/df_complete.csv")
df_complete = df_complete[df_complete.split == "train"]
plant_dataset = PlantTraitsDataset(
                df=df_complete,
                transform=datamodule.test_transform,
            )

In [ ]:
plant_loader = DataLoader(plant_dataset, batch_size=128, num_workers=8, shuffle=False)

In [ ]:
all_predictions = []
i = 0
for batch in tqdm(plant_loader):
    # Unpack the batch
    x, x_ = batch["image"].to(device), batch["metadata"].to(device)
    # Move data to the device

    # Make predictions
    with torch.no_grad():
        pred_enc = model.model.forward_alt(x, x_)
        pred = model.model.le.inverse_transform(pred_enc.clone().detach())
        pred = pred.cpu()
    # Append predictions to the list
    all_predictions.append(pred)
# Concatenate predictions from all batches
all_predictions = torch.concat(all_predictions, dim=0)

In [ ]:
all_predictions = all_predictions.numpy()

In [ ]:
trait_columns_pred = [i+"_pred" for i in trait_columns]
trait_columns_pred

In [ ]:
df_complete[trait_columns_pred] = all_predictions

In [ ]:
df_complete.shape

In [ ]:
df_complete[df_complete.split == "test"]

In [ ]:
df_complete.to_csv("/home/ubuntu/FGVC11/data/PlantTrait/df_complete.csv", index=False)

### Mapping to the closest values

In [ ]:
full_df = pd.read_csv('/home/ubuntu/FGVC11/data/PlantTrait/df_processed.csv')

In [ ]:
species_traits = full_df[trait_columns + ['species']][full_df.split != 'test'].groupby('species').mean()
species_traits.reset_index(inplace=True)

In [ ]:
submission_df = pd.read_csv("avg_v0_v_1_2.csv")

In [ ]:
mapped_traits = []
for _, row in tqdm(submission_df.iterrows(), total=len(submission_df)):
    obj_traits = []
    for col in trait_columns:
        col_sub = col.replace("_mean", "")
        pred_trt = row[col_sub]
        tgt_trts = species_traits[col]
        mapped_trait = tgt_trts[np.argmin(abs(pred_trt - tgt_trts))]
        obj_traits.append(mapped_trait)
        # break
    obj_traits = np.array(obj_traits)
    mapped_traits.append(obj_traits)

In [ ]:
mapped_traits = np.array(mapped_traits)

In [ ]:
# Create a DataFrame with the predictions and corresponding IDs
mapped_sub = pd.DataFrame({
    'id': submission_df['id'].values,
    'X4': mapped_traits[:, 0],
    'X11': mapped_traits[:, 1],
    'X18': mapped_traits[:, 2],
    'X50': mapped_traits[:, 3],
    'X26': mapped_traits[:, 4],
    'X3112': mapped_traits[:, 5],
})

In [ ]:
mapped_sub

In [ ]:
submission_df

In [ ]:
# for col in trait_columns:
#     col = col.replace("_mean", "")
#     print(f"r2 {col}: {r2_score(sub[col], mapped_sub[col])}")
best_submission = pd.read_csv("avg_v0_v_1_2.csv")
r2_score(torch.tensor(best_submission[best_submission.columns[1:]].values), torch.tensor(mapped_sub[best_submission.columns[1:]].values))

In [ ]:
mapped_sub.to_csv('mapped_sub.csv', index=False)

In [ ]:
!kaggle competitions submit -c planttraits2024 -f mapped_sub.csv -m "mapped_v0_v1_2"

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from fgvc.models.plant_traits_model import StructuredSelfAttention

class TraitBlender(nn.Module):
    def __init__(self, input_dim, output_dim, num_blocks=1, n_models=2, dropout_rate=0.1):
        super(TraitBlender, self).__init__()
        self.n_models = n_models
        # Adjust the input dimension based on the number of models (traits sets)
        self.self_attention = StructuredSelfAttention(input_dim * n_models, output_dim, num_blocks)
        self.dropout = nn.Dropout(dropout_rate)
        self.final_layer = nn.Linear(output_dim, input_dim)  # Output dimension aligns back to the number of traits

    def forward(self, traits_list):
        # Ensure we have the correct number of trait sets
        if len(traits_list) != self.n_models:
            raise ValueError(f"Expected {self.n_models} sets of traits, but got {len(traits_list)}")

        # Concatenate the input traits along the feature dimension
        combined_input = torch.cat(traits_list, dim=1)

        # Process combined input through the Structured Self-Attention
        attention_output = self.self_attention(combined_input)

        # Apply dropout and pass through the final linear layer
        dropped_output = self.dropout(attention_output)
        blended_traits = self.final_layer(dropped_output)

        return blended_traits

# Assuming the StructuredSelfAttention class is defined as previously
# Example usage:
n_models = 3  # Number of sets of traits to blend
num_features = 6  # Number of features per trait

trait_blender = TraitBlender(num_features, num_features * n_models, num_blocks=1, n_models=n_models, dropout_rate=0.1)

# Example tensors for multiple sets of pred_traits
pred_traits_1 = torch.rand(10, 6)
pred_traits_2 = torch.rand(10, 6)
pred_traits_3 = torch.rand(10, 6)

# Get the blended traits
blended_traits = trait_blender([pred_traits_1, pred_traits_2, pred_traits_3])


In [ ]:
blended_traits.shape